In [0]:
# genesis_workbench.dev_mmt_core_test

# REQUIRES Classic ML Compute?

In [0]:
catalog_name = "genesis_workbench"
schema_name = "dev_mmt_core_test"

In [0]:
# Construct the SQL query string
drop_schema_query = f"""
DROP SCHEMA IF EXISTS {catalog_name}.{schema_name} CASCADE;
"""

# create_schema_query = f"""
# CREATE SCHEMA {catalog_name}.{schema_name};
# """

# Execute the SQL queries
spark.sql(drop_schema_query)
# spark.sql(create_schema_query)

In [0]:
# full_schema = f"{catalog_name}.{schema_name}"
# tables_df = spark.sql(f"SHOW TABLES IN {full_schema}")
# tables = tables_df.collect()

In [0]:
# display(tables)

In [0]:
import os

# Set these environment variables
os.environ['DATABRICKS_HOST'] = 'https://adb-830292400663869.9.azuredatabricks.net/'
os.environ['DATABRICKS_TOKEN'] = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get() ## TO replace with Service Principal Token...

host = os.environ['DATABRICKS_HOST']
token = os.environ['DATABRICKS_TOKEN']

In [0]:
import mlflow

def delete_models_by_prefix_or_suffix(catalog, schema, prefix=None, suffix=None):
    # Set the registry URI to use Unity Catalog
    mlflow.set_registry_uri(f'databricks-uc://{catalog}.{schema}')
    
    # Search for all registered models
    models = mlflow.search_registered_models()
    
    models_to_delete = []
    
    for model in models:
        model_name = model.name
        
        # Check if the model name matches the prefix or suffix
        if (prefix and model_name.startswith(prefix)) or (suffix and model_name.endswith(suffix)):
            models_to_delete.append(model)
    
    if not models_to_delete:
        print(f"No models found with prefix '{prefix}' or suffix '{suffix}' in catalog '{catalog}' and schema '{schema}'")
        return
    
    print(f"Found {len(models_to_delete)} models with prefix '{prefix}' or suffix '{suffix}' in catalog '{catalog}' and schema '{schema}':")
    for model in models_to_delete:
        print(f"- Model: {model.name}")
        for version in model.latest_versions:
            print(f"  - Version: {version.version}")
    
    # Confirm before proceeding
    confirm = input(f"Delete all models with prefix '{prefix}' or suffix '{suffix}' in catalog '{catalog}' and schema '{schema}'? (yes/no): ")
    if confirm.lower() != 'yes':
        print("Deletion cancelled.")
        return
    
    for model in models_to_delete:
        model_name = model.name
        for version in model.latest_versions:
            mlflow.delete_model_version(name=model_name, version=version.version)
        mlflow.delete_registered_model(name=model_name)
        print(f"✓ Deleted model: {model_name}")

# # Example usage
# catalog_name = "genesis_workbench"
# schema_name = "dev_mmt_core_test"
# delete_models_by_prefix_or_suffix(catalog=catalog_name, schema=schema_name, prefix="scimilarity_")

In [0]:
# delete_serving_endpoints_with_prefix("gwb_scimilarity_")
delete_serving_endpoints_with_prefix("mmt_scimilarity")

In [0]:
import mlflow
import os

# Set MLflow to use Databricks backend
mlflow.set_tracking_uri("databricks")

# For Unity Catalog models, ensure proper authentication
mlflow.set_registry_uri("databricks-uc")

In [0]:
import mlflow
from mlflow.tracking.client import MlflowClient
from typing import Optional, List

def delete_model_versions(CATALOG, DB_SCHEMA, model_name, delete_all=False, specific_versions:Optional[List[int]]=None):
    """
    Delete all versions or specific versions of a registered model.

    Parameters:
    model_name (str): The name of the model to delete.
    delete_all (bool): If True, delete all versions of the model. If False, delete specific versions.
    specific_versions (list): List of specific versions to delete if delete_all is False.
    """
    full_model_name = f"{CATALOG}.{DB_SCHEMA}.{model_name}"

    # Initialize the MLflow client
    client = MlflowClient()

    # Fetch all versions of the model
    model_version_infos = client.search_model_versions(f"name='{full_model_name}'")

    if delete_all:
        # Delete all versions of the model
        for model_version_info in model_version_infos:
            client.delete_model_version(name=full_model_name, version=model_version_info.version)
    else:
        if specific_versions is None:
            raise ValueError("specific_versions must be provided if delete_all is False")
        # Delete specific versions of the model
        for version in specific_versions:
            client.delete_model_version(name=full_model_name, version=version)

    # Delete the registered model
    client.delete_registered_model(name=full_model_name)

    print(f"Deleted model '{full_model_name}' and all its versions.")

# Example usage
# delete_model_versions("SCimilarity_GeneOrder", delete_all=True)
# delete_model_versions("SCimilarity_GeneOrder", delete_all=False, specific_versions=["3", "4", "5"])

In [0]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

# List all registered models
models = client.search_registered_models()
# for model in models:
#     print(model.name)
[model.name for model in models if "mmt_" in model.name]

In [0]:
# delete_model_versions(catalog_name, schema_name, "scimilarity_gene_order", delete_all=True)
# delete_model_versions(catalog_name, schema_name, "scimilarity_get_embedding", delete_all=True)
# delete_model_versions(catalog_name, schema_name, "scimilarity_search_nearest", delete_all=True)